<a href="https://colab.research.google.com/github/Erwanlbv/OrientalismProject/blob/master/Importing_clm_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/Erwanlbv/OrientalismProject

Cloning into 'OrientalismProject'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 6), reused 24 (delta 4), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [ ]:
cd transformers

/content/transformers/examples/pytorch/language-modeling/transformers


In [ ]:
!pip install .

Processing /content/transformers/examples/pytorch/language-modeling/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3231485 sha256=d34a840cfc9c99bad438f28eaab81ae47ea43b4371a9ac895603d1e91c9a4b69
  Stored in directory: /tmp/pip-ephem-wheel-cache-yase8cvo/wheels/0a/32/59/1aa56aaf1728a7a766e782c97c893f2cc9904bdaab5d0dbda6
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation:

In [ ]:
cd examples/pytorch/language-modeling/

[Errno 2] No such file or directory: 'examples/pytorch/language-modeling/'
/content/transformers/examples/pytorch/language-modeling/transformers/examples/pytorch/language-modeling


In [ ]:
!pip install -r requirements.txt

In [ ]:
from datasets import load_dataset

dataset = load_dataset('text', 
                       data_files={'train': ['/content/OrientalismProject/descartes.txt',
                                             '/content/OrientalismProject/david_hume.txt',
                                             '/content/OrientalismProject/kant_critic_of_pure_reason',
                                             '/content/OrientalismProject/david_hume.txt',
                                             '/content/OrientalismProject/kant_fundamental_principles.txt',
                                             '/content/OrientalismProject/locke_an_essay_concerning_human_understanding.txt']}
                       )
dataset.save_to_disk('/content/dataset')

Using custom data configuration default-fd05778fe00e77d9


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-fd05778fe00e77d9/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset["train"][100:110]

{'text': ['conversation of other men teaches us; to which may be added as the',
  'fourth, the reading, not of all books, but especially of such as',
  'have been written by persons capable of conveying proper',
  'instruction, for it is a species of conversation we hold with their',
  'authors. And it seems to me that all the wisdom we in ordinary',
  'possess is acquired only in these four ways; for I do not class',
  'divine revelation among them, because it does not conduct us by',
  'degrees, but elevates us at once to an infallible faith.',
  '',
  'There have been, indeed, in all ages great minds who endeavoured to']}

In [ ]:
import os
from pathlib import Path

In [ ]:
def merge_texts(paths):
  all_texts = ''
  for path in paths:
    all_texts += open('/content/' + path, 'r').read()
  return all_texts

In [ ]:
paths = [str(x) for x in Path(".").glob('**/*.txt')]
paths.remove('requirements.txt')
paths

['OrientalismProject/locke_an_essay_concerning_human_understanding.txt',
 'OrientalismProject/kant_fundamental_principles.txt',
 'OrientalismProject/descartes.txt',
 'OrientalismProject/david_hume.txt',
 'OrientalismProject/kant_critic_of_pure_reason.txt']

In [ ]:
dataset_path = '/content/dataset/'

if not os.path.exists(dataset_path):
  os.mkdirs(dataset_path)

file = open(dataset_path + 'all_texts.txt', 'w')
file.write(merge_texts(paths))
file.close()

In [ ]:
model_path = '/content/model'
if not os.path.exists(path):
  os.makedirs(path)

In [ ]:
!python run_clm.py \
  --model_name_or_path gpt2 \
  --train_file /content/dataset/all_texts.txt \
  --do_train \
  --do_eval \
  --block_size 512 \
  --output_dir /content/model \
  --per_device_train_batch_size 2 \
  --overwrite_output_dir 

11/30/2021 20:46:26 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
11/30/2021 20:46:26 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Named entity recognition pipeline, passing in a specific model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
BEGGING_TEXT = "We generally gather knowledge with the help of some source of justification. Historically justification sometimes under such names as “reason to believe”, “evidence”. Now, epistemology questions the very foundation of justification"

In [ ]:
BEGGING_TEXT2 = "Epistemology is"

In [ ]:
text = pipeline(BEGGING_TEXT2, max_length=200)[0]['generated_text']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated_list = list(text)
while '\n' in generated_list:
  generated_list[generated_list.index('\n')] = ' '
"".join(generated_list)

'Epistemology is (according to the first general principle) a conception of a necessary necessity—it cannot be given how absolutely impossible we conceive it to be the cause of a phenomena. Consequently, an object of possibility is not a part of our conception, but an empirical empirical notion of phenomena, that is to say, we find it in the empirical nature of our conception in general—it is always empirical (although a purely internal representation—in which case it does not make cognitions, which is only the mode of cognition that is possible) and is always possible.  As the conception of a thing in general does not apply to a phenomenon through which we can take something for an object, it cannot be given how absolutely impossible it should be if I could not give a cause, for example, how absolutely impossible, I could only form the conception of my present state, that is, of causality in general'

In [ ]:
text = """Epistemology is, by no means, to be taken\nadvantage of as an essential and necessary principle: for any particular\nmanifesto would not deserve the appellation'metaphysical', inasmuch as it were\npossible to prove the unity of all intelligences, though\nit would be absurd to employ it as a regulative principle. The\nmetaphysic of religion is therefore a very narrow one, which has to do\nwith the possibility or impossibility of any particular science:\nand, as we shall observe, the ideal of the common understanding must\nnot be, as appears frequently in it, based on the archetypes of the archetypes\nof reason itself; while the philosophy of reason, on the other hand,\ncannot (as has already been shown) have to do with the archetypes of\nreason alone. The conception of a world in which reason was the\nsoul, and of the existence of an intelligible being"""
